# 5-2 : 초고속 검색 시스템 구축하기

# 1.

## Step1

In [ ]:
# Step 1: 데이터 불러오기

# 문자열 데이터 리스트 생성
# 이 리스트는 사전 순 정렬 상태이며, 이후 이진 탐색 기반 접두사 검색에 사용됩니다.
word_list = [
    "apple",      # 과일
    "appliance",  # 가전제품
    "banana",     # 과일
    "band",       # 밴드(음악, 머리끈 등)
    "bandana",    # 반다나 (머리장식)
    "can",        # 캔
    "candle",     # 양초
    "candy",      # 사탕
    "cap",        # 모자
    "cape"        # 망토
]

# 사전 순 정렬 확인
# 이미 정렬되어 있지만, 만약 정렬이 되어 있지 않다면 다음 코드로 정렬할 수 있습니다.
# word_list.sort()

# 확인 출력
print("Step 1: 정렬된 단어 리스트 로딩 완료")
print("단어 수:", len(word_list))
print("단어 목록:", word_list)

## Step2

In [31]:
# Step 2: 선형 탐색 함수
# string startswith 함수 사용
def linear_prefix_search(words, prefix):
    """
    주어진 단어 리스트에서 특정 접두사(prefix)로 시작하는 모든 단어를 찾아 반환합니다.
    시간 복잡도: O(n) - 모든 단어를 한 번씩 검사

    Parameters:
    words (list of str): 검색 대상 단어 리스트
    prefix (str): 접두사

    Returns:
    list of str: 접두사로 시작하는 단어들
    """
    result = []  # 결과를 저장할 리스트

    # 단어 리스트를 처음부터 끝까지 하나씩 확인
    for word in words:
        # 단어가 prefix로 시작하면 결과 리스트에 추가
        if word.startswith(prefix):
            result.append(word)

    return result

In [32]:
# 테스트
matches = linear_prefix_search(word_list, "ban")
print("선형 탐색 결과:", matches)

선형 탐색 결과: ['banana', 'band', 'bandana']


## Step3

- 오류수정 : 강의 자료에는 binary_prefix_search() 함수의 선언과 내용이 분리되어 있는 오류가 있었음

In [37]:
# Step 3: 이진 탐색 기반 접두사 탐색
def lower_bound(words, prefix):
    """
    접두사(prefix)보다 **크거나 같은 첫 번째 인덱스**를 찾는 이진 탐색 함수.
    예: words = ['apple', 'banana', 'cap'], prefix = 'ban'일 때,
        반환값은 'banana'가 위치한 인덱스 1
    """
    # 탐색 범위의 시작(left)과 끝(right)을 설정
    left, right = 0, len(words)  # right는 실제 마지막 인덱스 + 1

    # left가 right보다 작을 때만 반복 (탐색 범위가 남아 있을 때)
    while left < right:
        # 중간 지점 계산
        mid = (left + right) // 2
        # 중간 값이 prefix보다 작으면, 찾는 값은 오른쪽에 있음 → left를 mid + 1로 이동
        if words[mid] < prefix:
            left = mid + 1
        else:
            # 중간 값이 prefix 이상이면, mid도 후보이므로 right를 mid로 이동
            right = mid

    # left == right가 되는 순간, prefix 이상인 첫 번째 인덱스를 의미
    return left

def binary_prefix_search(words, prefix):
    """
    정렬된 문자열 리스트에서 이진 탐색을 활용해
    특정 접두사로 시작하는 모든 단어를 찾아 반환합니다.

    시간 복잡도: O(log n + k)
    - log n: 시작 인덱스 찾기 (이진 탐색)
    - k: 접두사로 시작하는 단어 개수

    Parameters:
    words (list of str): 사전 순으로 정렬된 단어 리스트
    prefix (str): 찾고자 하는 접두사

    Returns:
    list of str: prefix로 시작하는 단어들의 리스트
    """
    result = []
    start_index = lower_bound(words, prefix)

    # start_index부터 하나씩 검사하며 접두사가 일치하면 결과에 추가
    for i in range(start_index, len(words)):
        if words[i].startswith(prefix):
            result.append(words[i])
        else:
            break # 접두사가 일치하지 않는 순간 종료 (정렬되어 있으므로)

    return result

In [38]:
matches = binary_prefix_search(word_list, "ban")
print("이진 탐색 결과:", matches)

이진 탐색 결과: ['banana', 'band', 'bandana']


## Step4

In [35]:
# 전역 캐시 딕셔너리 선언 (접두사 → 결과 리스트)
search_cache = {}

def cached_search(prefix):
    """
    접두사 검색을 캐시를 활용하여 빠르게 처리하는 함수.
    - 캐시에 결과가 있으면 바로 반환
    - 없으면 binary_prefix_search로 결과 생성 후 저장

    Parameters:
    prefix (str): 검색할 접두사

    Returns:
    list of str: 접두사로 시작하는 단어들
    """
    # 1. 캐시에 prefix가 이미 존재하는 경우
    if prefix in search_cache:
        print(f"✅ [캐시 사용] '{prefix}' 결과 반환")
        return search_cache[prefix]

    # 2. 없으면 이진 탐색 함수로 결과 생성
    print(f"🔍 [이진 탐색 수행] '{prefix}' 검색 중...")
    result = binary_prefix_search(word_list, prefix)

    # 3. 캐시에 저장 후 반환
    search_cache[prefix] = result
    return result

In [36]:
# 처음 검색: 이진 탐색 수행
print(cached_search("ban"))

# 다시 검색: 캐시 사용
print(cached_search("ban"))

# 다른 접두사 검색
print(cached_search("cap"))

🔍 [이진 탐색 수행] 'ban' 검색 중...
['banana', 'band', 'bandana']
✅ [캐시 사용] 'ban' 결과 반환
['banana', 'band', 'bandana']
🔍 [이진 탐색 수행] 'cap' 검색 중...
['cap', 'cape']


## Step5

In [19]:
# 전역 검색 로그 리스트 선언
search_log = []

def log_search(prefix, result):
    """
    사용자의 검색 이력을 로그에 저장합니다.
    - 검색 접두사와 해당 결과를 튜플로 기록

    Parameters:
    prefix (str): 사용자가 입력한 접두사
    result (list of str): 접두사 검색 결과
    """
    # 튜플 형태로 (prefix, result) 저장
    search_log.append((prefix, result))

In [20]:
# 캐시 기능과 연계하여 사용
result1 = cached_search("ban")
log_search("ban", result1)

result2 = cached_search("cap")
log_search("cap", result2)

# 로그 출력
print("📜 검색 로그:")
for entry in search_log:
    print(entry)

✅ [캐시 사용] 'ban' 결과 반환
✅ [캐시 사용] 'cap' 결과 반환
📜 검색 로그:
('ban', ['banana', 'band', 'bandana'])
('cap', ['cap', 'cape'])


## Step6

In [24]:
# Step 6: 단순 해시 함수 직접 구현
def simple_hash(word, table_size):
    """
    문자열의 각 문자의 아스키 값을 모두 더한 후
    주어진 테이블 크기(table_size)로 나눈 나머지를 해시값으로 반환합니다.

    Parameters:
    word (str): 해시를 구할 문자열
    table_size (int): 해시 테이블의 크기 (버킷 수)

    Returns:
    int: 계산된 해시 인덱스
    """
    total = 0

    # 문자열의 각 문자를 순회하며 아스키 값 누적
    for ch in word:
        total += ord(ch)

    # 테이블 크기로 나눈 나머지를 해시값으로 반환
    return total % table_size

In [25]:
table_size = 10

words_to_hash = ["apple", "banana", "cap", "candle"]

for w in words_to_hash:
    h = simple_hash(w, table_size)
    print(f"단어: {w:8} → 해시값: {h}")

단어: apple    → 해시값: 0
단어: banana   → 해시값: 9
단어: cap      → 해시값: 8
단어: candle   → 해시값: 5


## Step7

In [22]:
# Step 7: 해시 테이블 삽입 (체이닝 방식)
hash_table = [[] for _ in range(10)]

def insert_to_hash_table(word):
    table_size = len(hash_table)  # 해시 테이블의 크기
    index = simple_hash(word, table_size)
    if word not in hash_table[index]:
        hash_table[index].append(word)
        print(f"'{word}' → 인덱스 {index}에 삽입 완료")
    else:
        print(f"'{word}'는 이미 인덱스 {index}에 존재")

for word in word_list:
    insert_to_hash_table(word)

'apple' → 인덱스 0에 삽입 완료
'appliance' → 인덱스 1에 삽입 완료
'banana' → 인덱스 9에 삽입 완료
'band' → 인덱스 5에 삽입 완료
'bandana' → 인덱스 9에 삽입 완료
'can' → 인덱스 6에 삽입 완료
'candle' → 인덱스 5에 삽입 완료
'candy' → 인덱스 7에 삽입 완료
'cap' → 인덱스 8에 삽입 완료
'cape' → 인덱스 9에 삽입 완료


In [23]:
words = ["apple", "banana", "candle", "cap", "cape", "band", "candy"]

for word in words:
    insert_to_hash_table(word)

print("\n 현재 해시 테이블 상태:")
for i, bucket in enumerate(hash_table):
    print(f"인덱스 {i}: {bucket}")

'apple'는 이미 인덱스 0에 존재
'banana'는 이미 인덱스 9에 존재
'candle'는 이미 인덱스 5에 존재
'cap'는 이미 인덱스 8에 존재
'cape'는 이미 인덱스 9에 존재
'band'는 이미 인덱스 5에 존재
'candy'는 이미 인덱스 7에 존재

 현재 해시 테이블 상태:
인덱스 0: ['apple']
인덱스 1: ['appliance']
인덱스 2: []
인덱스 3: []
인덱스 4: []
인덱스 5: ['band', 'candle']
인덱스 6: ['can']
인덱스 7: ['candy']
인덱스 8: ['cap']
인덱스 9: ['banana', 'bandana', 'cape']


## Step8

In [26]:
# Step 8: 해시 테이블을 이용한 단어 탐색
def find_in_hash_table(word):
    """
    주어진 단어가 해시 테이블에 존재하는지 여부를 반환합니다.
    simple_hash로 인덱스를 구한 뒤, 해당 인덱스 리스트에서 선형 탐색을 수행합니다.

    Parameters:
    word (str): 찾고자 하는 단어

    Returns:
    bool: 단어가 존재하면 True, 없으면 False
    """
    table_size = len(hash_table)  # 해시 테이블 크기
    index = simple_hash(word, table_size)  # 단어의 해시 인덱스 계산

    # 해당 인덱스(버킷)에 단어가 존재하는지 확인
    if word in hash_table[index]:
        print(f"'{word}'는 해시 인덱스 {index}에 존재합니다.")
        return True
    else:
        print(f"'{word}'는 해시 인덱스 {index}에 없습니다.")
        return False

In [27]:
print(find_in_hash_table("apple"))     # True
print(find_in_hash_table("appliance")) # False (삽입 안 했음)
print(find_in_hash_table("cap"))       # True
print(find_in_hash_table("ghost"))     # False

'apple'는 해시 인덱스 0에 존재합니다.
True
'appliance'는 해시 인덱스 1에 존재합니다.
True
'cap'는 해시 인덱스 8에 존재합니다.
True
'ghost'는 해시 인덱스 9에 없습니다.
False


## Step9

- 오류 수정 : optimized_search() 함수가 정의되어 있지 않았음

In [28]:
# Step 9: 최적화 검색 (통합)
def optimized_search(prefix):
    """
    최적화된 접두사 검색 함수입니다.

    이 함수는 아래 3가지 기능을 통합하여 처리합니다:

    1. 캐시(cache) 사용:
       - 이전에 검색한 접두사는 해시 테이블(dict)을 통해 즉시 결과 반환 (O(1))
       - 처음 보는 접두사는 binary_prefix_search()를 통해 검색 후 캐시에 저장

    2. 검색 결과 로그(log) 저장:
       - 사용자가 어떤 prefix를 검색했는지, 그 결과는 무엇이었는지를 search_log에 기록

    3. 결과 반환:
       - 최종적으로 사용자에게 접두사로 시작하는 단어 리스트를 반환

    Parameters:
    prefix (str): 사용자가 입력한 검색 접두사

    Returns:
    list of str: prefix로 시작하는 모든 단어 리스트
    """
    # 1. 캐시 검색 또는 이진 탐색 실행
    result = cached_search(prefix)

    # 2. 검색 이력 로그에 저장
    log_search(prefix, result)

    # 3. 검색 결과 반환
    return result


In [29]:
# 테스트 케이스 실행
print(optimized_search("ban"))
print(optimized_search("cap"))
print(optimized_search("ban"))  # 캐시 HIT 확인
print(optimized_search("app"))

# 최종 검색 로그 확인
print("\n📜 최종 검색 로그:")
for idx, entry in enumerate(search_log, 1):
    print(f"{idx}. {entry}")

✅ [캐시 사용] 'ban' 결과 반환
['banana', 'band', 'bandana']
✅ [캐시 사용] 'cap' 결과 반환
['cap', 'cape']
✅ [캐시 사용] 'ban' 결과 반환
['banana', 'band', 'bandana']
🔍 [이진 탐색 수행] 'app' 검색 중...
['apple', 'appliance']

📜 최종 검색 로그:
1. ('ban', ['banana', 'band', 'bandana'])
2. ('cap', ['cap', 'cape'])
3. ('ban', ['banana', 'band', 'bandana'])
4. ('cap', ['cap', 'cape'])
5. ('ban', ['banana', 'band', 'bandana'])
6. ('app', ['apple', 'appliance'])


## Step10

In [30]:
# Step 10: 전체 시나리오 실행

print("\n[시나리오 1] 'ban' 검색 → 결과 출력")
result1 = optimized_search("ban")
print("🔹 결과:", result1)

print("\n[시나리오 2] 'app' 검색 → 결과 출력")
result2 = optimized_search("app")
print("🔹 결과:", result2)

print("\n[시나리오 3] 'ban' 재검색 → 캐시에서 탐색 확인")
result3 = optimized_search("ban")
print("🔹 결과:", result3)

print("\n[시나리오 4] 전체 검색 로그 출력")
for idx, entry in enumerate(search_log, 1):
    prefix, results = entry
    print(f"{idx}. 접두사: '{prefix}' → 결과: {results}")


[시나리오 1] 'ban' 검색 → 결과 출력
✅ [캐시 사용] 'ban' 결과 반환
🔹 결과: ['banana', 'band', 'bandana']

[시나리오 2] 'app' 검색 → 결과 출력
✅ [캐시 사용] 'app' 결과 반환
🔹 결과: ['apple', 'appliance']

[시나리오 3] 'ban' 재검색 → 캐시에서 탐색 확인
✅ [캐시 사용] 'ban' 결과 반환
🔹 결과: ['banana', 'band', 'bandana']

[시나리오 4] 전체 검색 로그 출력
1. 접두사: 'ban' → 결과: ['banana', 'band', 'bandana']
2. 접두사: 'cap' → 결과: ['cap', 'cape']
3. 접두사: 'ban' → 결과: ['banana', 'band', 'bandana']
4. 접두사: 'cap' → 결과: ['cap', 'cape']
5. 접두사: 'ban' → 결과: ['banana', 'band', 'bandana']
6. 접두사: 'app' → 결과: ['apple', 'appliance']
7. 접두사: 'ban' → 결과: ['banana', 'band', 'bandana']
8. 접두사: 'app' → 결과: ['apple', 'appliance']
9. 접두사: 'ban' → 결과: ['banana', 'band', 'bandana']


# 2.

###1) simple_hash 대신 더 정교한 해시 함수로 성능 개선

In [44]:
import random
import string
from collections import defaultdict

# 🔹 단순 해시 함수 (원본)
def simple_hash(word, table_size):
    return sum(ord(ch) for ch in word) % table_size

# 🔹 개선된 해시 함수 (djb2)
def better_hash(word, table_size):
    hash_value = 5381
    for ch in word:
        hash_value = ((hash_value << 5) + hash_value) + ord(ch)  # hash * 33 + ch
    return hash_value % table_size

# 🔹 무작위 단어 생성 함수
def generate_random_words(n, length=6):
    return [''.join(random.choices(string.ascii_lowercase, k=length)) for _ in range(n)]

# 🔹 충돌 횟수 계산 함수
def test_collision_rate(hash_func, words, table_size):
    hash_table = defaultdict(list)
    collisions = 0
    for word in words:
        index = hash_func(word, table_size)
        if hash_table[index]:  # 버킷이 비어 있지 않으면 충돌
            collisions += 1
        hash_table[index].append(word)
    return collisions

# 🔹 테스트 실행
if __name__ == "__main__":
    table_size = 100
    test_words = generate_random_words(1000)  # 1000개 단어 생성

    simple_collisions = test_collision_rate(simple_hash, test_words, table_size)
    better_collisions = test_collision_rate(better_hash, test_words, table_size)

    print(f"🔍 simple_hash 충돌 횟수: {simple_collisions}")
    print(f"✅ better_hash(djb2) 충돌 횟수: {better_collisions}")


🔍 simple_hash 충돌 횟수: 909
✅ better_hash(djb2) 충돌 횟수: 900


### 2) 입력 리스트를 10만 개 이상으로 확장해 성능 비교

In [40]:
import random
import string
import time

# 무작위 단어 생성기
def generate_random_words(n, length=6):
    return [''.join(random.choices(string.ascii_lowercase, k=length)) for _ in range(n)]

# 테스트용 단어 리스트 생성
large_word_list = sorted(generate_random_words(100000))  # 정렬된 상태 유지

# 성능 측정
def test_binary_search_performance(words, prefix):
    start = time.time()
    binary_prefix_search(words, prefix)
    end = time.time()
    print(f"[{prefix}] 검색 소요 시간: {end - start:.6f}초")

# 테스트 실행
test_binary_search_performance(large_word_list, "ab")
test_binary_search_performance(large_word_list, "xyz")


[ab] 검색 소요 시간: 0.000060초
[xyz] 검색 소요 시간: 0.000011초


### 3) 검색 결과에 자동 완성 기능 추가 (가장 많이 검색된 단어 추천)

In [46]:
from collections import Counter

# 🔹 추천 접두사 추출 (search_log 기반)
def recommend_prefixes(log, top_n=5):
    """
    자주 검색된 접두사 top-N 추천

    Parameters:
    log (list): (prefix, result) 로그
    top_n (int): 추천 개수

    Returns:
    list of (prefix, count): 접두사와 검색 횟수
    """
    counter = Counter(prefix for prefix, _ in log)
    return counter.most_common(top_n)


# 🔹 자동완성 단어 추천
def autocomplete_words(prefix, words, max_n=5):
    """
    접두사로 시작하는 단어 추천

    Parameters:
    prefix (str): 사용자 입력
    words (list): 전체 단어 리스트 (정렬된 상태)
    max_n (int): 추천 단어 최대 개수

    Returns:
    list: 추천 단어 리스트
    """
    results = binary_prefix_search(words, prefix)
    return results[:max_n]


# 🔹 사용자 출력 예시
def print_recommendations(prefix, word_list, search_log):
    print(f"\n✅ 입력한 접두사: '{prefix}'")

    # 자동완성 단어 출력
    print("🔎 자동완성 추천 단어:")
    for word in autocomplete_words(prefix, word_list):
        print(f"  • {word}")

    # 인기 검색 접두사 추천
    print("\n🔥 인기 검색 접두사:")
    for p, cnt in recommend_prefixes(search_log):
        print(f"  • '{p}' ({cnt}회 검색됨)")


In [47]:
# 예시 시나리오: 검색 기록
search_log = [
    ("ban", ["banana", "band", "bandana"]),
    ("cap", ["cap", "cape"]),
    ("ban", ["banana", "band", "bandana"]),
    ("app", ["apple", "appliance"]),
    ("app", ["apple", "appliance"]),
    ("app", ["apple", "appliance"]),
]

# 정렬된 단어 리스트
word_list = sorted(["apple", "appliance", "banana", "band", "bandana", "cap", "cape", "candle", "candy"])

# 실행
print_recommendations("ap", word_list, search_log)



✅ 입력한 접두사: 'ap'
🔎 자동완성 추천 단어:
  • apple
  • appliance

🔥 인기 검색 접두사:
  • 'app' (3회 검색됨)
  • 'ban' (2회 검색됨)
  • 'cap' (1회 검색됨)
